In [3]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 25.1/25.1 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.1
    Uninstalling pyarrow-14.0.1:
      Successfully uninstalled pyarrow-14.0.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch

# 모델 및 토크나이저 로드
model_name = "microsoft/deberta-v3-base"  # DeBERTa 모델
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)
model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터 전처리 함수 정의
def preprocess_text(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=128)

# 데이터셋 로드 및 전처리
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['text'], train_data['target'], test_size=0.2, random_state=42
)

# 토큰화 및 변환
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Hugging Face 데이터셋 포맷으로 변환
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': list(train_labels)
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': list(val_labels)
})

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)

# 평가 메트릭 함수 정의
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 모델 학습
trainer.train()

# 검증 데이터 예측
results = trainer.evaluate()
print("Validation Accuracy:", results["eval_accuracy"])

# 테스트 데이터 예측 및 제출 파일 생성
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, max_length=128)
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
})

# 예측
predictions = trainer.predict(test_dataset)
test_data['target'] = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

# 제출 파일 생성
submission = test_data[['id', 'target']]
submission.to_csv('submission_v5_DeBERTa_v3.csv', index=False)
print("Submission file saved as 'submission.csv'")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-effde27965d2>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.367400,0.381325,0.835850
2,0.335900,0.432938,0.835850
3,0.371000,0.456798,0.838477


Validation Accuracy: 0.8384767174720764
Submission file saved as 'submission.csv'


In [14]:
!pip uninstall pyarrow -y


Found existing installation: pyarrow 18.0.0
Uninstalling pyarrow-18.0.0:
  Successfully uninstalled pyarrow-18.0.0


In [15]:
!pip install pyarrow --force-reinstall


Defaulting to user installation because normal site-packages is not writeable
  Using cached pyarrow-18.0.0-cp39-cp39-win_amd64.whl.metadata (3.4 kB)
Using cached pyarrow-18.0.0-cp39-cp39-win_amd64.whl (25.1 MB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
!conda install -c conda-forge pyarrow

^C


In [2]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   --------------------------------------- 991.5/991.5 kB 15.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# 그리드 서치

from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch

# 모델 및 토크나이저 로드
model_name = "microsoft/deberta-v3-base"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)

# 데이터셋 로드
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["text"], train_data["target"], test_size=0.2, random_state=42
)

# 데이터 전처리 함수 정의
def preprocess_data(texts, labels=None):
    encodings = tokenizer(
        list(texts),
        truncation=True,
        padding=True,
        max_length=128,
    )
    if labels is not None:
        encodings["labels"] = list(labels)
    return encodings

# 데이터 토큰화
train_encodings = preprocess_data(train_texts, train_labels)
val_encodings = preprocess_data(val_texts, val_labels)

# Hugging Face 데이터셋 포맷으로 변환
train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

# 평가 메트릭 함수 정의
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), axis=1)
    accuracy = (predictions == torch.tensor(labels)).float().mean()
    return {"accuracy": accuracy.item()}

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

C:\Users\yukyu\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yukyu\.cache\huggingface\hub\models--microsoft--deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

C:\Users\yukyu\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
# 하이퍼파라미터 그리드 정의
grid = {
    "learning_rate": [2e-5],
    "per_device_train_batch_size": [8],
    "num_train_epochs": [2, 3]
}

In [3]:
# 최적의 하이퍼파라미터를 저장할 변수
best_accuracy = 0
best_params = {}

# 그리드 서치 실행
for learning_rate in grid["learning_rate"]:
    for batch_size in grid["per_device_train_batch_size"]:
        for num_epochs in grid["num_train_epochs"]:
            print(f"Testing config: lr={learning_rate}, batch_size={batch_size}, epochs={num_epochs}")

            # 학습 설정
            training_args = TrainingArguments(
                output_dir="./results",
                evaluation_strategy="epoch",
                save_strategy="no",  # 모델 저장 생략
                num_train_epochs=num_epochs,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                learning_rate=learning_rate,
                logging_dir="./logs",
                logging_steps=10,
                report_to="none"
            )

            # 모델 초기화
            model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=2)

            # Trainer 설정
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
            )

            # 모델 학습
            trainer.train()

            # 검증 데이터 평가
            results = trainer.evaluate()
            accuracy = results["eval_accuracy"]
            print(f"Accuracy: {accuracy}")

            # 최적의 하이퍼파라미터 업데이트
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    "learning_rate": learning_rate,
                    "batch_size": batch_size,
                    "num_train_epochs": num_epochs,
                }

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:")
print(best_params)
print(f"Best Validation Accuracy: {best_accuracy}")

Testing config: lr=2e-05, batch_size=8, epochs=2


C:\Users\yukyu\AppData\Roaming\Python\Python39\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# 최적의 하이퍼파라미터로 모델 재학습
print("Training with Best Hyperparameters...")
best_learning_rate = best_params["learning_rate"]
best_batch_size = best_params["batch_size"]
best_num_epochs = best_params["num_train_epochs"]

# 최적의 하이퍼파라미터로 학습 설정
final_training_args = TrainingArguments(
    output_dir="./final_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=best_num_epochs,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    learning_rate=best_learning_rate,
    logging_dir="./final_logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)

# 최종 모델 초기화
final_model = DebertaV2ForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 최종 Trainer 설정
final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 최종 모델 학습
final_trainer.train()

# 테스트 데이터 예측
print("Predicting on Test Data...")
test_encodings = preprocess_data(test_data["text"])
test_dataset = Dataset.from_dict(test_encodings)

test_predictions = final_trainer.predict(test_dataset)
test_data["target"] = torch.argmax(torch.tensor(test_predictions.predictions), axis=1).numpy()

# 제출 파일 생성
submission = test_data[["id", "target"]]
submission.to_csv("submission_v5_DeBERTa_GridSearch.csv", index=False)
print("Submission file saved as 'submission.csv'")